# Created an interactive map for CRESST/GAIA webpage 

This map is to provide context for UW research teams working on geohazards studies in Washington State. It is intented to show where in-situ stations are in relation to various observational and modeling products as well as admin and physical features such as roads, rivers, etc. The map does not provide real-time data streams, but it does provide links to APIs to access data for analysis.

Adding new layers:
1. Add a new markdown section if needed 
2. Add a layer with a unique name 
3. Ensure your layer is added to the TreeLayerControl at the end

Design goals:
- do not embed data if possible (use APIs to load any data on-demand)
- use standard / existing colormaps suggested by data providers
- keep it simple :) 

In [ ]:
import folium
import geopandas as gpd
import urllib.parse
import requests
import branca
from folium.elements import MacroElement
from jinja2 import Template
import xyzservices
import numpy as np
import matplotlib
import matplotlib.colors as mcolors
import branca.colormap as bcm

In [ ]:
# Restrict map to WA State
aoi = gpd.read_file(
    "https://raw.githubusercontent.com/unitedstates/districts/refs/heads/gh-pages/states/WA/shape.geojson"
)
minlon, minlat, maxlon, maxlat = aoi.total_bounds

In [ ]:
m = folium.Map(
    min_lat= minlat - 1,
    max_lat= maxlat + 1,
    min_lon= minlon - 1,
    max_lon= maxlon + 1,
    max_bounds=True,
    # # NOTE: something seems to be override this...
    min_zoom=7,
    max_zoom=12,
    zoom_start=7,
    control_scale = True,
)

# Basemaps

To see what is out there: https://xyzservices.readthedocs.io/en/stable/

In [ ]:
# Simplify adding tiles w/ XYZ services
#xyzservices.providers.Esri#.WorldImagery

In [ ]:
# Keep track of layer name and layer in a dictionayr
basemap_layers = {}

# Default Basemap = CartoDB Tiles
# =============
tiles = folium.TileLayer(
    tiles=xyzservices.providers.CartoDB.Positron,
    overlay=False,
    show=True,
    control=True
).add_to(m)

basemap_layers['CartoDB'] = tiles

# ESRI Tiles
# =============
#ESRI_API="https://server.arcgisonline.com/ArcGIS/rest/services/{variant}/MapServer/tile/{z}/{y}/{x}"
for variant in ['WorldImagery','WorldShadedRelief','WorldTopoMap','WorldGrayCanvas']:
    layer_name = f"ESRI {variant}"
    tiles = folium.TileLayer(
        tiles=xyzservices.providers.Esri[variant],
        name=layer_name,
        overlay=False,
        show=False,
        control=True
    ).add_to(m)

    basemap_layers[layer_name] = tiles

# NASA GIBS
# =============
# A lot of available layers, just using DEM for starters
# https://worldview.earthdata.nasa.gov
# https://www.earthdata.nasa.gov/engage/open-data-services-software/earthdata-developer-portal/gibs-api
layer_name = "ASTER GDEM Greyscale Shaded Relief"
tiles = folium.raster_layers.WmsTileLayer(
    url="https://gibs.earthdata.nasa.gov/wms/epsg3857/best/wms.cgi",
    layers="ASTER_GDEM_Greyscale_Shaded_Relief",
    name=layer_name,
    fmt="image/png",
    transparent=True,
    overlay=False,
    control=True,
    show=False,
).add_to(m)

basemap_layers[layer_name] = tiles

# Example for mult-year prodcts
#years = [2023,2024,2025]
years = [2025]
for year in years:
    layer_name = f"OPERA Annual Disturbance {year}"
    tiles = folium.raster_layers.WmsTileLayer(
        #url="https://gibs.earthdata.nasa.gov/wms/epsg3857/best/wms.cgi",
        url=f"https://gibs.earthdata.nasa.gov/wms/epsg3857/best/wms.cgi?TIME={year}-01-01",
        layers="OPERA_L3_DIST-ANN-HLS_Color_Index",
        name=layer_name,
        fmt="image/png",
        #transparent=True,
        overlay=False,
        control=True,
        show=False,
        # NOT working.... need to pass in URL param directly
        #extra_wms_params={
        #    "TIME": "2023-01-01"
            #"TIME": "2024-06-01T00:00:00Z"
        #},
    ).add_to(m)

    basemap_layers[layer_name] = tiles

# TODO: legends for sophisticated layers
# https://gibs.earthdata.nasa.gov/wms/epsg3857/best/?request=GetMetadata&layer=OPERA_L3_DIST-ANN-HLS_Color_Index
# Apparently has a legend URL! https://gibs.earthdata.nasa.gov/legends/OPERA_Vegetation_Disturbance_Annual_H.png
# legend_url = "https://gibs.earthdata.nasa.gov/legends/OPERA_Vegetation_Disturbance_Annual_H.png"
# legend_html = f"""
# <div style="
#     position: fixed;
#     bottom: 50px; left: 50px; width: 300px; z-index:9999;
#     background-color: white; padding: 10px; border:2px solid grey;
#     box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">
#     <b>OPERA Annual Disturbance 2024 Legend</b><br>
#     <img src="{legend_url}" style="width:100%;"/>
# </div>
# """
# m.get_root().html.add_child(Element(legend_html))

## GIS / Vector Layers

### WA DNR Fire polygons

In [ ]:
# url = "https://gis.dnr.wa.gov/site3/rest/services/Public_Wildfire/WADNR_PUBLIC_WD_WildFire_Data/MapServer/0/query"
# params = {
#     #'where': 'YEAR = 2024',
#     'where': 'YEAR >= 2023',
#     #'where': 'YEAR >= 2021',   #> maxRecordCount
#     'outFields': '*',
#     'f': 'geojson'
# }
# query_string = urllib.parse.urlencode(params)
# full_url = f"{url}?{query_string}"
# print(full_url)
# #fires = gpd.read_file(full_url)

# gjson = folium.GeoJson(full_url,
#                         embed=False,
#                         show=False,
#                         name="WA DNR Wildfire boundaries (>=2023)",
#                         style_function=lambda x: {
#                             'fillColor': 'none',
#                             'color': 'red',
#                             'weight': 2,
#                             'fillOpacity': 0
#                         }).add_to(m)

# all_attributes = ['OBJECTID', 'UNITID', 'AGENCY', 'FIRENAME', 'FIRENUM', 'STARTDATE', 'PERIMDATE', 'ACRES', 'YEAR', 'CAUSE', 'SHAPE.AREA', 'SHAPE.LEN', 'geometry']
# show_attributes = ['FIRENAME', 'YEAR', 'STARTDATE', 'PERIMDATE', 'SHAPE.AREA',  'CAUSE']
# folium.features.GeoJsonTooltip(fields=["FIRENAME"], labels=False).add_to(gjson)
# folium.features.GeoJsonPopup(fields=show_attributes, labels=True).add_to(gjson);

### Google Roads

In [ ]:
gis_layers = {}
# Add Google Roads layer
layer_name = 'Google Roads'
tiles_roads = folium.raster_layers.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=h&x={x}&y={y}&z={z}',
    attr='Google',
    name=layer_name,
    overlay=True,
    control=True,
    show=False,
).add_to(m);

gis_layers[layer_name] = tiles_roads

### HUC8 Watersheds

In [ ]:
# Watersheds
# Get GeoJSON from an API
huc8 = 'https://hydro.nationalmap.gov/arcgis/rest/services/wbd/FeatureServer/4/query?where=1%3D1&geometry=-124%2C45%2C-116%2C49&geometryType=esriGeometryEnvelope&inSR=4326&spatialRel=esriSpatialRelIntersects&outFields=name%2Chuc8%2Careasqkm&outSR=4326&f=geojson&maxAllowableOffset=0.01'

gjson = folium.GeoJson(huc8,
                        embed=False,
                        show=False,
                        name="Watershed Sub-basins (HUC8)",
                        style_function=lambda x: {
                            'fillColor': 'blue', # 'none' = no fill. if no fill, no hover tips
                            'color': 'blue',
                            'weight': 2,
                            'fillOpacity': 0.1, #0 = totally transpatent
                        }).add_to(m)

gis_layers['Watershed Sub-basins (Huc8)'] = gjson

folium.features.GeoJsonTooltip(fields=["name"], labels=False).add_to(gjson)
folium.features.GeoJsonPopup(fields=["name", "huc8", "areasqkm"], labels=True).add_to(gjson);

## Raster Layers 


### Dynamic tiling of tiffs

We'll rely on https://titiler.xyz to render public rasters on-the-fly


for these we need a colorbar to interpret the rendered tiff...

In [ ]:
def mpl_to_branca(cmap, vmin=0, vmax=1, n=256):
    """
    Convert a matplotlib colormap to a branca LinearColormap.

    Parameters
    ----------
    cmap : matplotlib colormap or str
        Colormap instance or name (e.g. 'viridis')
    vmin, vmax : float
        Data range for the branca colormap
    n : int
        Number of color samples

    Returns
    -------
    branca.colormap.LinearColormap
    """
    if isinstance(cmap, str):
        cmap = matplotlib.colormaps.get_cmap(cmap)

    colors = [
        mcolors.to_hex(cmap(i))
        for i in np.linspace(0, 1, n)
    ]

    return bcm.LinearColormap(
        colors=colors,
        vmin=vmin,
        vmax=vmax
    )

In [ ]:
# Colorbar for titiler raster overlays - works with TreeLayerControl
class ToggleableLayerColorbar(MacroElement):
    """
    A colorbar legend that toggles visibility based on layer add/remove events.
    Works with TreeLayerControl by tracking the layer object directly.
    """
    _counter = 0  # Class-level counter for unique IDs

    def __init__(self, layer, colormap):
        """
        Parameters
        ----------
        layer : folium layer object
            The layer to associate with this colorbar (e.g., a TileLayer)
        colormap : branca colormap
            The colormap to display
        """
        super().__init__()
        self.layer = layer
        self.colormap = colormap
        ToggleableLayerColorbar._counter += 1
        self.legend_var_name = f"layer_legend_{ToggleableLayerColorbar._counter}"

        self._template = Template("""
        {% macro script(this, kwargs) %}
        // Initialize registry if not exists
        if (typeof window._layerColorbarLegends === 'undefined') {
            window._layerColorbarLegends = {};
        }

        var {{ this.legend_var_name }} = L.control({position: 'bottomright'});
        {{ this.legend_var_name }}.onAdd = function (map) {
            var div = L.DomUtil.create('div', 'info legend');
            div.style.backgroundColor = 'white';
            div.style.padding = '10px';
            div.innerHTML = `{{ this.colormap._repr_html_() }}`;
            return div;
        };
        // Don't add to map initially - will be added when layer is shown
        {{ this.legend_var_name }}._isOnMap = false;

        // Get reference to the layer
        var targetLayer_{{ this.legend_var_name }} = {{ this.layer.get_name() }};

        // Register this legend with the layer reference
        window._layerColorbarLegends['{{ this.legend_var_name }}'] = {
            legend: {{ this.legend_var_name }},
            layer: targetLayer_{{ this.legend_var_name }}
        };

        // Listen for layer add events on the map
        {{ this._parent.get_name() }}.on('layeradd', function(e) {
            if (e.layer === targetLayer_{{ this.legend_var_name }}) {
                // Remove all other legends from the map first
                for (var key in window._layerColorbarLegends) {
                    var entry = window._layerColorbarLegends[key];
                    if (entry.legend._isOnMap) {
                        entry.legend.remove();
                        entry.legend._isOnMap = false;
                    }
                }
                // Add this legend to the map
                {{ this.legend_var_name }}.addTo({{ this._parent.get_name() }});
                {{ this.legend_var_name }}._isOnMap = true;
            }
        });

        // Listen for layer remove events on the map
        {{ this._parent.get_name() }}.on('layerremove', function(e) {
            if (e.layer === targetLayer_{{ this.legend_var_name }}) {
                if ({{ this.legend_var_name }}._isOnMap) {
                    {{ this.legend_var_name }}.remove();
                    {{ this.legend_var_name }}._isOnMap = false;
                }
            }
        });
        {% endmacro %}
        """)

### SOLUS Soil properties

In [ ]:
# Keep track of all COG layers for Dynamic Tiling
cog_layers = {}

tif = "https://storage.googleapis.com/solus100pub/anylithicdpt_cm_p.tif"
tiler = "https://titiler.xyz/cog/tiles/WebMercatorQuad/{z}/{x}/{y}"
layer_name = 'SOLUS100 Soil Depth'
virtual_tiles = f"{tiler}?url={tif}"
tiles_solus = folium.TileLayer(tiles=virtual_tiles,
                 overlay=True,
                 control=True,
                 show=False,
                 name=layer_name,
                 attr="USDA").add_to(m)

cog_layers[layer_name] = tiles_solus

cm_solus = branca.colormap.linear.Greys_06.scale(0,1)
cm_solus.colors.reverse()
cm_solus = branca.colormap.LinearColormap(colors=cm_solus.colors,
                                          caption='SOLUS100 soil depth (cm)',
                                          vmin=0, vmax=200)

# Link colorbar visibility to layer visibility
#m.add_child(ToggleableColorbar(layer_name, cm));
m.add_child(ToggleableLayerColorbar(tiles_solus, cm_solus));

### Seismic Velocities

In [ ]:
# Instead of COG approach, use USGS MapServer Directly...
#tif = "https://dasway.ess.washington.edu/shared/niyiyu/global_vs30_2.tif"
#tiler = "https://titiler.xyz/cog/tiles/WebMercatorQuad/{z}/{x}/{y}"
#virtual_tiles = f"{tiler}?url={tif}&rescale={vmin},{vmax}&colormap_name={colormap}"

layer_name = 'USGS Vs30 Mosaic'
tiles_vs30 = folium.TileLayer(
    tiles='https://earthquake.usgs.gov/arcgis/rest/services/eq/vs30_mosaic/MapServer/tile/{z}/{y}/{x}',
    attr='USGS',
    name=layer_name,
    overlay=True,
    control=True,
    show=False,
).add_to(m)

cog_layers[layer_name] = tiles_vs30

In [ ]:
legend_response = requests.get('https://earthquake.usgs.gov/arcgis/rest/services/eq/vs30_mosaic/MapServer/legend?f=json')
legend_data = legend_response.json()

# Create a toggleable legend for the Vs30 layer using ESRI legend JSON
class ToggleableEsriLegend(MacroElement):
    """
    A legend that toggles visibility based on layer add/remove events.
    Builds HTML legend from ESRI MapServer legend JSON response.
    """
    _counter = 0

    def __init__(self, layer, legend_json, title="Legend"):
        """
        Parameters
        ----------
        layer : folium layer object
            The layer to associate with this legend
        legend_json : dict
            The JSON response from ESRI MapServer legend endpoint
        title : str
            Title to display at the top of the legend
        """
        super().__init__()
        self.layer = layer
        self.legend_json = legend_json
        self.title = title
        ToggleableEsriLegend._counter += 1
        self.legend_var_name = f"esri_legend_{ToggleableEsriLegend._counter}"

        # Build the legend HTML from the JSON
        self.legend_html = self._build_legend_html()

        self._template = Template("""
        {% macro script(this, kwargs) %}
        // Initialize registry if not exists
        if (typeof window._layerEsriLegends === 'undefined') {
            window._layerEsriLegends = {};
        }

        var {{ this.legend_var_name }} = L.control({position: 'bottomright'});
        {{ this.legend_var_name }}.onAdd = function (map) {
            var div = L.DomUtil.create('div', 'info legend');
            div.style.backgroundColor = 'white';
            div.style.padding = '10px';
            div.style.maxHeight = '300px';
            div.style.overflowY = 'auto';
            div.innerHTML = `{{ this.legend_html }}`;
            return div;
        };
        {{ this.legend_var_name }}._isOnMap = false;

        var targetLayer_{{ this.legend_var_name }} = {{ this.layer.get_name() }};

        window._layerEsriLegends['{{ this.legend_var_name }}'] = {
            legend: {{ this.legend_var_name }},
            layer: targetLayer_{{ this.legend_var_name }}
        };

        {{ this._parent.get_name() }}.on('layeradd', function(e) {
            if (e.layer === targetLayer_{{ this.legend_var_name }}) {
                // Remove all colorbar legends first
                if (typeof window._layerColorbarLegends !== 'undefined') {
                    for (var key in window._layerColorbarLegends) {
                        var entry = window._layerColorbarLegends[key];
                        if (entry.legend._isOnMap) {
                            entry.legend.remove();
                            entry.legend._isOnMap = false;
                        }
                    }
                }
                // Remove all ESRI legends
                for (var key in window._layerEsriLegends) {
                    var entry = window._layerEsriLegends[key];
                    if (entry.legend._isOnMap) {
                        entry.legend.remove();
                        entry.legend._isOnMap = false;
                    }
                }
                {{ this.legend_var_name }}.addTo({{ this._parent.get_name() }});
                {{ this.legend_var_name }}._isOnMap = true;
            }
        });

        {{ this._parent.get_name() }}.on('layerremove', function(e) {
            if (e.layer === targetLayer_{{ this.legend_var_name }}) {
                if ({{ this.legend_var_name }}._isOnMap) {
                    {{ this.legend_var_name }}.remove();
                    {{ this.legend_var_name }}._isOnMap = false;
                }
            }
        });
        {% endmacro %}
        """)

    def _build_legend_html(self):
        """Build HTML string from ESRI legend JSON."""
        html = f'<b>{self.title}</b><br>'

        if 'layers' in self.legend_json and len(self.legend_json['layers']) > 0:
            legend_items = self.legend_json['layers'][0].get('legend', [])
            for item in legend_items:
                label = item.get('label', '')
                image_data = item.get('imageData', '')
                content_type = item.get('contentType', 'image/png')

                html += f'''
                <div style="display: flex; align-items: center; margin: 2px 0;">
                    <img src="data:{content_type};base64,{image_data}"
                         style="width: 20px; height: 20px; margin-right: 5px;"/>
                    <span style="font-size: 12px;">{label}</span>
                </div>'''

        return html

# Add the toggleable legend for Vs30 layer
m.add_child(ToggleableEsriLegend(tiles_vs30, legend_data, title="USGS Vs30 (m/s)"));

### Atmospheric Observations (Precip, ERM)

In [ ]:
## For rainfall accumulation

# Set up scaling and tile rendering
layer_name_bk = "Stage IV Precip, Dec. 2025"
attribution = "NOAA"
colormap = "viridis"
vmin = 0
vmax = 700
cmap_stage4 = mpl_to_branca(colormap, vmin=vmin, vmax=vmax)
cmap_stage4.caption = "Stage IV Precipitation (mm)"

tif_bk = "https://gaia-hazlab-map-data.s3.us-west-2.amazonaws.com/rainfall_4km.tif"
tiler_bk = "https://titiler.xyz/cog/tiles/WebMercatorQuad/{z}/{x}/{y}"
virtual_tiles_bk = f"{tiler_bk}?url={tif_bk}&rescale={vmin},{vmax}&nodata=nan&colormap_name={colormap}"

# Display the colormap
cmap_stage4

In [ ]:
tiles_stage4 =folium.TileLayer(
    tiles=virtual_tiles_bk,
    overlay=True,
    control=True,
    show=False,
    name=layer_name_bk,
    attr=attribution,
    opacity=0.6
).add_to(m)

cog_layers[layer_name_bk] = tiles_stage4

m.add_child(ToggleableLayerColorbar(tiles_stage4, cmap_stage4));

In [ ]:
## For max ERM

# Set up scaling and tile rendering
layer_name_bk2 = "Max ERM, Dec. 2025"
attribution = "UW Atmospheric Sciences"
colormap = "plasma"
vmin = 0
vmax = 3
cmap_erm = mpl_to_branca(colormap, vmin=vmin, vmax=vmax)
cmap_erm.caption = "Max Extreme Rainfall Multiplier (ERM)"

tif_bk2 = "https://gaia-hazlab-map-data.s3.us-west-2.amazonaws.com/max_erm_4km.tif"
tiler_bk2 = "https://titiler.xyz/cog/tiles/WebMercatorQuad/{z}/{x}/{y}"
virtual_tiles_bk2 = f"{tiler_bk2}?url={tif_bk2}&rescale={vmin},{vmax}&nodata=nan&colormap_name={colormap}"

cmap_erm

In [ ]:
tiles_erm = folium.TileLayer(
    tiles=virtual_tiles_bk2,
    overlay=True,
    control=True,
    show=False,
    name=layer_name_bk2,
    attr=attribution,
    opacity=0.6
).add_to(m)

cog_layers[layer_name_bk2] = tiles_erm

m.add_child(ToggleableLayerColorbar(tiles_erm, cmap_erm));

## Stations

Add these last so that hovering always shows station info (rather than HUC polygon info)

In [ ]:
# Keep track of layer name and layer in a dictionary
station_layers = {}

### Seismometers

From https://ds.iris.edu/ds/nodes/dmc/

In [ ]:
layer_name = "Seismometers"
url = "https://raw.githubusercontent.com/gaia-hazlab/catalog/refs/heads/main/seismic-stations.geojson"
gf = gpd.read_file(url)
all_fields = list(gf.drop(columns='geometry').columns)
#color = gf['marker-color'].iloc[0]
color = "#FF00BB"  # magenta color for seismic

# TODO: Experiments with different icons... but these are all too small when zoomed in...

# Better dynamic scaling when zoomed in?
# size=8
# myicon = folium.DivIcon(
#         html=f'''
#             <svg width="{size}" height="{size}" viewBox="0 0 24 24">
#                 <polygon points="12,20 4,4 20,4" fill="{color}"/>
#             </svg>
#         ''',
#         icon_size=(size, size),
#         icon_anchor=(size/2, size/2)
#    )
# From https://icons.getbootstrap.com
size=8
myicon = folium.DivIcon(
        html=f'''
            <svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="{color}" class="bi bi-caret-down" viewBox="0 0 16 16">
            <path d="M3.204 5h9.592L8 10.481zm-.753.659 4.796 5.48a1 1 0 0 0 1.506 0l4.796-5.48c.566-.647.106-1.659-.753-1.659H3.204a1 1 0 0 0-.753 1.659"/>
            </svg>
        ''',
        icon_size=(size, size),
        icon_anchor=(size/2, size/2)
   )

# PNGs
# size = 8
# myicon = folium.CustomIcon(
#     #icon_image='https://img.icons8.com/?size=100&id=37218&format=png&color=000000',  # You'd need an image file
#     icon_image=f'https://img.icons8.com/?size=100&id=10767&format=png&color={color.strip("#")}',
#     icon_size=(size, size)
# )

# Upside down triangle for seismic stations
# size = 3
# myicon = folium.DivIcon(html=f'<div style="border-left:{size}px solid transparent; border-right:{size}px solid transparent; border-top:{size*2}px solid {color};"></div>',
#                             icon_size=(size*2, size*2), # Width = 2*size px, Height = 2*size px
#                             icon_anchor=(size, 0)      # Center horizontally, anchor at top
#     )

# Ensure that stations are always on top for hoverInfo by adding them after all other layers and using bringToFront()
gjson = folium.GeoJson(url,
                        embed=False,
                        show=False,
                        marker=folium.Marker(icon=myicon),
                        #marker=folium.Circle(fill=False, radius=200, color=color),
                        name=layer_name).add_to(m)

station_layers[layer_name] = gjson

folium.features.GeoJsonTooltip(fields=['station'], labels=False).add_to(gjson)
folium.features.GeoJsonPopup(fields=all_fields, labels=True).add_to(gjson);

### Infrasound

In [ ]:
layer_name = "Infrasound"
url = "https://raw.githubusercontent.com/gaia-hazlab/catalog/refs/heads/main/infrasound-stations.geojson"
gf = gpd.read_file(url)
all_fields = list(gf.drop(columns='geometry').columns)
color = "#1A982F"  # green color for infrasound

# Mic ICON!
size=8
myicon = folium.DivIcon(
        html=f'''
<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="{color}" class="bi bi-mic" viewBox="0 0 16 16">
  <path d="M3.5 6.5A.5.5 0 0 1 4 7v1a4 4 0 0 0 8 0V7a.5.5 0 0 1 1 0v1a5 5 0 0 1-4.5 4.975V15h3a.5.5 0 0 1 0 1h-7a.5.5 0 0 1 0-1h3v-2.025A5 5 0 0 1 3 8V7a.5.5 0 0 1 .5-.5"/>
  <path d="M10 8a2 2 0 1 1-4 0V3a2 2 0 1 1 4 0zM8 0a3 3 0 0 0-3 3v5a3 3 0 0 0 6 0V3a3 3 0 0 0-3-3"/>
</svg>
        ''',
        icon_size=(size, size),
        icon_anchor=(size/2, size/2)
   )

# Ensure that stations are always on top for hoverInfo by adding them after all other layers and using bringToFront()
gjson = folium.GeoJson(url,
                        embed=False,
                        show=False,
                        marker=folium.Marker(icon=myicon),
                        #marker=folium.Circle(fill=False, radius=200, color=color),
                        name=layer_name).add_to(m)

station_layers[layer_name] = gjson

folium.features.GeoJsonTooltip(fields=['station'], labels=False).add_to(gjson)
folium.features.GeoJsonPopup(fields=all_fields, labels=True).add_to(gjson);

### Synoptic API Stations

### Snotel

In [ ]:
layer_name = "SNOTEL"
# TODO: use consistent hyphens vs underscors...
url = "https://raw.githubusercontent.com/gaia-hazlab/catalog/refs/heads/main/snotel_stations.geojson"
gf = gpd.read_file(url)
all_fields = list(gf.drop(columns='geometry').columns)
color = "#04E9E9"  # blue color for SNOTEL

# SNOW ICON!
size=8
myicon = folium.DivIcon(
        html=f'''
<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="{color}" class="bi bi-snow2" viewBox="0 0 16 16">
  <path d="M8 16a.5.5 0 0 1-.5-.5v-1.293l-.646.647a.5.5 0 0 1-.707-.708L7.5 12.793v-1.086l-.646.647a.5.5 0 0 1-.707-.708L7.5 10.293V8.866l-1.236.713-.495 1.85a.5.5 0 1 1-.966-.26l.237-.882-.94.542-.496 1.85a.5.5 0 1 1-.966-.26l.237-.882-1.12.646a.5.5 0 0 1-.5-.866l1.12-.646-.884-.237a.5.5 0 1 1 .26-.966l1.848.495.94-.542-.882-.237a.5.5 0 1 1 .258-.966l1.85.495L7 8l-1.236-.713-1.849.495a.5.5 0 1 1-.258-.966l.883-.237-.94-.542-1.85.495a.5.5 0 0 1-.258-.966l.883-.237-1.12-.646a.5.5 0 1 1 .5-.866l1.12.646-.237-.883a.5.5 0 0 1 .966-.258l.495 1.849.94.542-.236-.883a.5.5 0 0 1 .966-.258l.495 1.849 1.236.713V5.707L6.147 4.354a.5.5 0 1 1 .707-.708l.646.647V3.207L6.147 1.854a.5.5 0 1 1 .707-.708l.646.647V.5a.5.5 0 0 1 1 0v1.293l.647-.647a.5.5 0 1 1 .707.708L8.5 3.207v1.086l.647-.647a.5.5 0 1 1 .707.708L8.5 5.707v1.427l1.236-.713.495-1.85a.5.5 0 1 1 .966.26l-.236.882.94-.542.495-1.85a.5.5 0 1 1 .966.26l-.236.882 1.12-.646a.5.5 0 0 1 .5.866l-1.12.646.883.237a.5.5 0 1 1-.26.966l-1.848-.495-.94.542.883.237a.5.5 0 1 1-.26.966l-1.848-.495L9 8l1.236.713 1.849-.495a.5.5 0 0 1 .259.966l-.883.237.94.542 1.849-.495a.5.5 0 0 1 .259.966l-.883.237 1.12.646a.5.5 0 0 1-.5.866l-1.12-.646.236.883a.5.5 0 1 1-.966.258l-.495-1.849-.94-.542.236.883a.5.5 0 0 1-.966.258L9.736 9.58 8.5 8.866v1.427l1.354 1.353a.5.5 0 0 1-.707.708l-.647-.647v1.086l1.354 1.353a.5.5 0 0 1-.707.708l-.647-.647V15.5a.5.5 0 0 1-.5.5"/>
</svg>
        ''',
        icon_size=(size, size),
        icon_anchor=(size/2, size/2)
   )

# Ensure that stations are always on top for hoverInfo by adding them after all other layers and using bringToFront()
gjson = folium.GeoJson(url,
                        embed=False,
                        show=False,
                        marker=folium.Marker(icon=myicon),
                        #marker=folium.Circle(fill=False, radius=200, color=color),
                        name=layer_name).add_to(m)

station_layers[layer_name] = gjson

folium.features.GeoJsonTooltip(fields=['stid'], labels=False).add_to(gjson)
folium.features.GeoJsonPopup(fields=all_fields, labels=True).add_to(gjson);

### Precipitation

In [ ]:
layer_name = "Precipitation"
url = "https://raw.githubusercontent.com/gaia-hazlab/catalog/refs/heads/main/precip-stations.geojson"
gf = gpd.read_file(url)
all_fields = list(gf.drop(columns='geometry').columns)
color = "#1B04E9"  # blue color for Precipitation

# Droplet ICON!
size=8
myicon = folium.DivIcon(
        html=f'''
<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="{color}" class="bi bi-droplet" viewBox="0 0 16 16">
  <path fill-rule="evenodd" d="M7.21.8C7.69.295 8 0 8 0q.164.544.371 1.038c.812 1.946 2.073 3.35 3.197 4.6C12.878 7.096 14 8.345 14 10a6 6 0 0 1-12 0C2 6.668 5.58 2.517 7.21.8m.413 1.021A31 31 0 0 0 5.794 3.99c-.726.95-1.436 2.008-1.96 3.07C3.304 8.133 3 9.138 3 10a5 5 0 0 0 10 0c0-1.201-.796-2.157-2.181-3.7l-.03-.032C9.75 5.11 8.5 3.72 7.623 1.82z"/>
  <path fill-rule="evenodd" d="M4.553 7.776c.82-1.641 1.717-2.753 2.093-3.13l.708.708c-.29.29-1.128 1.311-1.907 2.87z"/>
</svg>
        ''',
        icon_size=(size, size),
        icon_anchor=(size/2, size/2)
   )

# Ensure that stations are always on top for hoverInfo by adding them after all other layers and using bringToFront()
gjson = folium.GeoJson(url,
                        embed=False,
                        show=True,
                        marker=folium.Marker(icon=myicon),
                        #marker=folium.Circle(fill=False, radius=200, color=color),
                        name=layer_name).add_to(m)

station_layers[layer_name] = gjson

folium.features.GeoJsonTooltip(fields=['stid'], labels=False).add_to(gjson)
folium.features.GeoJsonPopup(fields=all_fields, labels=True).add_to(gjson);

### Streamflow

In [ ]:
layer_name = "Streamflow"
url = "https://raw.githubusercontent.com/gaia-hazlab/catalog/refs/heads/main/streamflow-stations.geojson"
gf = gpd.read_file(url)
all_fields = list(gf.drop(columns='geometry').columns)
color = "#E9AC04"  # yellow color for Streamflow

# Flow/water ICON!
size=8
myicon = folium.DivIcon(
        html=f'''
<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="{color}" class="bi bi-water" viewBox="0 0 16 16">
  <path d="M.036 3.314a.5.5 0 0 1 .65-.278l1.757.703a1.5 1.5 0 0 0 1.114 0l1.014-.406a2.5 2.5 0 0 1 1.857 0l1.015.406a1.5 1.5 0 0 0 1.114 0l1.014-.406a2.5 2.5 0 0 1 1.857 0l1.015.406a1.5 1.5 0 0 0 1.114 0l1.757-.703a.5.5 0 1 1 .372.928l-1.758.703a2.5 2.5 0 0 1-1.857 0l-1.014-.406a1.5 1.5 0 0 0-1.114 0l-1.015.406a2.5 2.5 0 0 1-1.857 0l-1.014-.406a1.5 1.5 0 0 0-1.114 0l-1.015.406a2.5 2.5 0 0 1-1.857 0L.314 3.964a.5.5 0 0 1-.278-.65m0 3a.5.5 0 0 1 .65-.278l1.757.703a1.5 1.5 0 0 0 1.114 0l1.014-.406a2.5 2.5 0 0 1 1.857 0l1.015.406a1.5 1.5 0 0 0 1.114 0l1.014-.406a2.5 2.5 0 0 1 1.857 0l1.015.406a1.5 1.5 0 0 0 1.114 0l1.757-.703a.5.5 0 1 1 .372.928l-1.758.703a2.5 2.5 0 0 1-1.857 0l-1.014-.406a1.5 1.5 0 0 0-1.114 0l-1.015.406a2.5 2.5 0 0 1-1.857 0l-1.014-.406a1.5 1.5 0 0 0-1.114 0l-1.015.406a2.5 2.5 0 0 1-1.857 0L.314 6.964a.5.5 0 0 1-.278-.65m0 3a.5.5 0 0 1 .65-.278l1.757.703a1.5 1.5 0 0 0 1.114 0l1.014-.406a2.5 2.5 0 0 1 1.857 0l1.015.406a1.5 1.5 0 0 0 1.114 0l1.014-.406a2.5 2.5 0 0 1 1.857 0l1.015.406a1.5 1.5 0 0 0 1.114 0l1.757-.703a.5.5 0 1 1 .372.928l-1.758.703a2.5 2.5 0 0 1-1.857 0l-1.014-.406a1.5 1.5 0 0 0-1.114 0l-1.015.406a2.5 2.5 0 0 1-1.857 0l-1.014-.406a1.5 1.5 0 0 0-1.114 0l-1.015.406a2.5 2.5 0 0 1-1.857 0L.314 9.964a.5.5 0 0 1-.278-.65m0 3a.5.5 0 0 1 .65-.278l1.757.703a1.5 1.5 0 0 0 1.114 0l1.014-.406a2.5 2.5 0 0 1 1.857 0l1.015.406a1.5 1.5 0 0 0 1.114 0l1.014-.406a2.5 2.5 0 0 1 1.857 0l1.015.406a1.5 1.5 0 0 0 1.114 0l1.757-.703a.5.5 0 1 1 .372.928l-1.758.703a2.5 2.5 0 0 1-1.857 0l-1.014-.406a1.5 1.5 0 0 0-1.114 0l-1.015.406a2.5 2.5 0 0 1-1.857 0l-1.014-.406a1.5 1.5 0 0 0-1.114 0l-1.015.406a2.5 2.5 0 0 1-1.857 0l-1.757-.703a.5.5 0 0 1-.278-.65"/>
</svg>
        ''',
        icon_size=(size, size),
        icon_anchor=(size/2, size/2)
   )

# Ensure that stations are always on top for hoverInfo by adding them after all other layers and using bringToFront()
gjson = folium.GeoJson(url,
                        embed=False,
                        show=False,
                        marker=folium.Marker(icon=myicon),
                        #marker=folium.Circle(fill=False, radius=200, color=color),
                        name=layer_name).add_to(m)

station_layers[layer_name] = gjson

folium.features.GeoJsonTooltip(fields=['stid'], labels=False).add_to(gjson)
folium.features.GeoJsonPopup(fields=all_fields, labels=True).add_to(gjson);

### Precip and Streamflow (old)

From https://synopticdata.com

In [ ]:
# # Add In-Situ Stations
# stations = {'Precipitation': "https://raw.githubusercontent.com/gaia-hazlab/catalog/refs/heads/main/precip-stations-wa-styled.geojson",
#             'Streamflow': "https://raw.githubusercontent.com/gaia-hazlab/catalog/refs/heads/main/streamflow-stations-wa-styled.geojson"}

# for name, url in stations.items():

#     # Get list of columns for popup
#     gf = gpd.read_file(url)
#     drop_cols = ['marker-size', 'marker-symbol', 'marker-color', 'geometry']
#     all_fields = list(gf.drop(columns=drop_cols).columns)
#     color = gf['marker-color'].iloc[0]
#     hover_label = 'stid'

#     mymarker = folium.Circle(fill=False, radius=200, color=color)

#     # Ensure that stations are always on top for hoverInfo by adding them after all other layers and using bringToFront()
#     gjson = folium.GeoJson(url,
#                            embed=False,
#                            marker=mymarker,
#                            name=f"{name}").add_to(m)

#     station_layers[name] = gjson

#     folium.features.GeoJsonTooltip(fields=[hover_label], labels=False).add_to(gjson)
#     folium.features.GeoJsonPopup(fields=all_fields, labels=True).add_to(gjson);

In [ ]:
# Ensure stations are always on top when any overlay is toggled via TreeLayerControl
m.keep_in_front(*station_layers.values())

## Optional Plugins

- Ability to measure distances 
- Ability to draw vector shapes to export (e.g. study area AOI)

In [ ]:
# Optional plugins
from folium.plugins import Draw
from folium.plugins import MeasureControl

# Position options position : {'topleft', 'toprigth', 'bottomleft', 'bottomright'}
# https://github.com/python-visualization/folium/issues/1806
Draw(export=True, position='bottomleft').add_to(m)

m.add_child(MeasureControl(position='topleft'));

## Layer Control / Legend

In [ ]:
from folium.plugins.treelayercontrol import TreeLayerControl

In [ ]:
basemap_tree = {
        "label": "Basemaps",
        "select_all_checkbox": False,
        "children": [{"label": key, "layer":val.add_to(m)} for key,val in basemap_layers.items()]
        }

overlay_tree = {
    "label": "All Layers",
    "select_all_checkbox": "Un/select all",
    "children": [
        {
        "label": "In Situ Sensors",
        "select_all_checkbox": True,
        "children": [{"label": key, "layer":val.add_to(m)} for key,val in station_layers.items()]
        },
        # TODO: turn into radioGroup so that only one can be selected at a time?
        # But we can have transparency, so maybe there's a way to stack colorbars...
        {
        "label": "Raster Layers",
        "select_all_checkbox": False,
        "children": [{"label": key, "layer":val.add_to(m)} for key,val in cog_layers.items()]
        },
        {
        "label": "GIS Layers",
        "select_all_checkbox": True,
        "children": [{"label": key, "layer":val.add_to(m)} for key,val in gis_layers.items()]
        },
    ]
    }

# Standard layer control
#folium.LayerControl(collapsed=True, draggable=True).add_to(m)
# Better Layer grouping and management:
TreeLayerControl(base_tree=basemap_tree, overlay_tree=overlay_tree).add_to(m);

## Display and save map

In [ ]:
# Preview the map in Jupyter Notebook to make sure things work as expected
m

In [ ]:
# This is what shows up on the website!
m.save('index.html')